# Problem Set 8

This problem set solves a matching problem in a marriage market. 

## Q1

\begin{align}
&E[\max \{\log(y+w_i) - \beta + \epsilon_1, \log(y)+ \epsilon_0 \}] \\
&=\sigma_\epsilon E[\max \{\dfrac{\log(y+w_i) - \beta + \epsilon_1}{\sigma_\epsilon} ,  \dfrac{\log(y)+ \epsilon_0}{\sigma_\epsilon} \}] \\
&= \sigma_\epsilon \ln[\dfrac{\log(y+w_i)-\beta}{\sigma_\epsilon} + \dfrac{\log(y)}{\sigma_\epsilon}] + \sigma_\epsilon \gamma
\end{align}

# Q2

### (a) Determin $s_{ij}$

Derive first order condition: 

\begin{align}
\lambda_{ij}\dfrac{1}{s_{ij} c} c + (1-\lambda_{ij}) \dfrac{1}{(1-s_{ij}) c} (-c) = 0
\end{align}

which gives us:
\begin{align}
s_{ij} = \lambda_{ij}
\end{align}

### (b) Expected utility of a type-i man in a type-ij marriage

Let $(x,y)$ denotes the employment status of the two spouses $(P_i,P_j)$, the first number represents man's employment, while the latter number is for women's. Then there are four cases of the man's utility:
* (0,0): $U = \log(2s_{ij}y) + \epsilon_{0,0}$
* (0,1): $U = \log(s_{ij}(2y+w_j)) + \epsilon_{0,1}$
* (1,0): $U = \log(s_{ij}(2y+w_i)) -\beta + \epsilon_{1,0}$
* (1,1): $U = \log(s_{ij}(2y+w_i+w_j)) - \beta + \epsilon_{1,1}$

Therefore, the expected utility is:
\begin{align}
\sigma_\epsilon\ln\left(exp(\dfrac{\log(2s_{ij}y)}{\sigma_\epsilon}) + exp(\dfrac{\log(s_{ij}(2y+w_i))}{\sigma_\epsilon}) + exp(\dfrac{\log(s_{ij}(2y+w_j))-\beta}{\sigma_\epsilon}) + exp(\dfrac{\log(s_{ij}(2y+w_i+w_j))-\beta}{\sigma_\epsilon}) \right)  + \sigma_\epsilon \gamma
\end{align}

## Q3

In [30]:
import numpy as np
from scipy import optimize
from scipy.optimize import brentq,fsolve


In [31]:
β = 0.8
σ_ϵ = 0.4
σ_θ = 0.7
γ = 0.5772
y = 1
w = np.array([1,2,3])

In [52]:
def prob_d(i,j,weight_matrix,τ):
    s_ij = weight_matrix[i,j]
    v_ij = σ_ϵ*(np.log(np.exp(np.log(2*s_ij*y)/σ_ϵ) + np.exp(np.log(s_ij*(2*y+w[i]))/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[j]))-β)/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[i]+w[j]))-β)/σ_ϵ) ) ) + σ_ϵ*γ
    den = np.exp(v_ij/σ_θ)
    num_temp = σ_ϵ*(np.log(np.exp((np.log(y+w[i]*(1-τ))-β)/σ_ϵ) + np.exp(np.log(y)/σ_ϵ))) + σ_ϵ*γ
    num = np.exp(num_temp/σ_θ)
    for j in range(3):
        s_ij = weight_matrix[i,j]
        v_j = σ_ϵ*(np.log(np.exp(np.log(2*s_ij*y)/σ_ϵ) + np.exp(np.log(s_ij*(2*y+w[i]))/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[j]))-β)/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[i]+w[j]))-β)/σ_ϵ) ) ) + σ_ϵ*γ
        numerator = np.exp(v_j/σ_θ)
        num = num + numerator
    return den/num

In [53]:
def prob_s(i,j,weight_matrix,τ):
    s_ij = 1 - weight_matrix[i,j]
    v_ji =  σ_ϵ*(np.log(np.exp(np.log(2*s_ij*y)/σ_ϵ) + np.exp(np.log(s_ij*(2*y+w[j]))/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[i]))-β)/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[j]+w[i]))-β)/σ_ϵ) ) ) + σ_ϵ*γ
    den = np.exp(v_ji/σ_θ)
    num_temp = σ_ϵ*(np.log(np.exp((np.log(y+w[j]*(1-τ))-β)/σ_ϵ) + np.exp(np.log(y)/σ_ϵ))) + σ_ϵ*γ
    num = np.exp(num_temp/σ_θ)
    for i in range(3):
        s_ij = 1 - weight_matrix[i,j]
        v_i = σ_ϵ*(np.log(np.exp(np.log(2*s_ij*y)/σ_ϵ) + np.exp(np.log(s_ij*(2*y+w[i]))/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[j]))-β)/σ_ϵ) + np.exp((np.log(s_ij*(2*y+w[i]+w[j]))-β)/σ_ϵ) ) ) + σ_ϵ*γ
        numerator =  np.exp(v_i/σ_θ)
        num = num + numerator
    return den/num

In [54]:

g = np.zeros([9])
τ = 0

def f(vector):
    λ = vector.reshape(3,3)
    n = 0
    for i in range(3):
        for j in range(3):
            g[n] = prob_d(i,j,λ,τ) - prob_s(i,j,λ,τ)
            n = n+1
    return g

In [57]:
weight = fsolve(f,(np.zeros([9])+0.2)).reshape(3,3)

C:\Users\zhuang13\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\zhuang13\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':


In [58]:
weight

array([[0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2]])